# Interactive Visualization of Daytimes and Workingtimes in Europe

## Data

In [2]:
import pandas as pd
import geopandas as gpd

RELOAD_DATA = True
eu_data_path = 'datasets/saved/eu_gpd.geojson'
city_data_path = 'datasets/saved/city_data.csv'
sunset_data = 'datasets/saved/sunset_data_2022.geojson'

### EU countries to capitals and GeoPandas data

In [4]:
from sun_data import get_sunset_sunrise_data
from geo_utils import load_eu_countries_as_geopandas, get_eu_city_data

if RELOAD_DATA:
    print('Reloading data from datasets/saved folder ...')
    eu_gpd = gpd.read_file(eu_data_path)
    top_city_data = pd.read_csv(city_data_path)
    sun_data_gpd = gpd.read_file(sunset_data)
else:
    print('Generating data ...')
    eu_gpd = load_eu_countries_as_geopandas()
    eu_gpd.to_file(eu_data_path, driver='GeoJSON', index=False)

    top_city_data = get_eu_city_data(3)
    top_city_data.to_csv(city_data_path, index=False)

    sun_data_gpd = get_sunset_sunrise_data(year=2022, eu_gpd=eu_gpd, csv_path=sunset_data)

Reloading data from datasets/saved folder ...


In [5]:
eu_gpd.head()

,pop_est,continent,name,iso_a3,gdp_md_est,iso_a2,geometry
0,5347896.0,Europe,Norway,NOR,403336,NO,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
1,67059887.0,Europe,France,FRA,2715518,FR,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
2,10285453.0,Europe,Sweden,SWE,530883,SE,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
3,9466856.0,Europe,Belarus,BLR,63080,BY,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."
4,44385155.0,Europe,Ukraine,UKR,153781,UA,"POLYGON ((32.15944 52.06125, 32.41206 52.28869..."


In [4]:
top_city_data.head()

,population,CODE,country_ISO_A2,NAME,longitude,latitude,mercantor_x,mercantor_y,social_timezone,utc_sun_timezone_offset,longitudinal_diff,longitudinal_diff_km
0,1205492,BE001C,BE,Bruxelles/Brussel,4.351697,50.846557,4.844287e+05,6.594196e+06,Europe/Brussels,1.0,10.648303,851.864240
1,523591,BE002C,BE,Antwerpen,4.399708,51.221110,4.897733e+05,6.660499e+06,Europe/Brussels,1.0,10.600292,848.023352
2,383710,BE005C,BE,Liège,5.573611,50.645094,6.204516e+05,6.558754e+06,Europe/Brussels,1.0,9.426389,754.111104
3,1238438,BG001C,BG,Sofia,23.321736,42.697703,2.596164e+06,5.266072e+06,Europe/Sofia,2.0,6.678264,534.261128
4,345213,BG002C,BG,Plovdiv,24.749930,42.141854,2.755150e+06,5.182252e+06,Europe/Sofia,2.0,5.250070,420.005624


In [5]:
sun_data_gpd.head()

,iso_a3,capital,day,month,year,sunrise_UTC,sunset_UTC,geometry
0,ALB,Tirana,1,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
1,ALB,Tirana,2,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
2,ALB,Tirana,3,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
3,ALB,Tirana,4,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."
4,ALB,Tirana,5,1,2022,6,15,"POLYGON ((2339940.185 4989171.529, 2337708.178..."


### Generate top n cities per country with timezone features

## Interactive visualization with Panel/Bokeh

In [6]:
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, ColumnDataSource, LabelSet, HoverTool
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Plasma256
from bokeh.palettes import brewer
import json
import panel as pn
from datetime import datetime, date

data_field = 'sunrise_UTC'
bokeh_tools = 'wheel_zoom, pan, box_zoom, reset'
colorbar_settings = {'title_text_font_size':'12pt','label_standoff':12}

def get_bokeh_geodata_source(gpd_df):
    json_data = json.dumps(json.loads(gpd_df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(data):
    p = figure(toolbar_location='right', tools=bokeh_tools, active_scroll ="wheel_zoom",
               title="Distance to eastern timezone meridian for large EU cities",
               x_range=(top_city_data['mercantor_x'].min(),top_city_data['mercantor_x'].max()))
    p.title.text_font_size = '20px'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # AD MAP TILES ---------------------------------------------------------------------------
    p.add_tile(Vendors.CARTODBPOSITRON_RETINA)

    # ADD GEO STUFF FOR COUNTRIES AS A WHOLE -------------------------------------------------
    geo_data_source = get_bokeh_geodata_source(data)

    values = data[data_field]
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = values.min(), high = values.max())
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), title='Sunrise (UTC)', **colorbar_settings)
    country_suntimes = p.patches('xs','ys', source=geo_data_source,
            fill_color={'field': data_field, 'transform': color_mapper},
            line_color='blue',
            line_width=0.5,
            fill_alpha=0.8)
    p.add_layout(color_bar, 'below')

      # TOOLTIPS FOR COUNTRY PATCHES
    tooltips_country = [
        ('Country', '@iso_a3'),
        ('Sunrise (UTC)', '@sunrise_UTC'),
        ('Sunset (UTC)', '@sunset_UTC')
    ]
    p.add_tools(HoverTool(renderers=[country_suntimes], tooltips=tooltips_country))

    # ===================================================================================================================

    #ADD TIMEZONE INFO PER TOP N CITIES FOR EACH COUNTRY ------------------------------------

    city_data_source = ColumnDataSource(data=top_city_data)
    city_color_mapper = LinearColorMapper(palette=Plasma256,
                                          low=top_city_data['longitudinal_diff_km'].min(),
                                          high=top_city_data['longitudinal_diff_km'].max())
    city_color_bar = ColorBar(color_mapper=city_color_mapper, location=(0,0),
                              title="Distance to east timezone meridian (Km)", **colorbar_settings)
    city_longdiff_circles = p.circle(x='mercantor_x', y='mercantor_y', source=city_data_source,
             color={'field': 'longitudinal_diff_km', 'transform': city_color_mapper}, size=10, fill_alpha=1)
    labels = LabelSet(x='mercantor_x', y='mercantor_y', x_offset=5, y_offset=5, text='NAME', source=city_data_source, text_color='cornflowerblue')

    p.add_layout(labels)
    p.add_layout(city_color_bar, 'right')

    # TOOLTIPS FOR CITY DATA
    tooltips_city = [
        ('Country', '@country_ISO_A2'),
        ('City', '@NAME'),
        ('Relative position (dist) to timezone border', '@longitudinal_diff_km')
    ]
    p.add_tools(HoverTool(renderers=[city_longdiff_circles], tooltips=tooltips_city, name='test'))
    return p

In [45]:
from bokeh.models import DataTable, TableColumn
from sklearn.preprocessing import MinMaxScaler

def bokeh_country_table(city_data, eu_data):
    # Add group city data by country
    cmeans =  city_data.groupby('country_ISO_A2')['longitudinal_diff_km'].mean()
    country_data = city_data.groupby('country_ISO_A2').first().reset_index()
    country_data['mean_longitudinal_diff_km'] = country_data.apply(lambda x: cmeans[x['country_ISO_A2']], axis=1)
    country_data = country_data[['social_timezone', 'mean_longitudinal_diff_km', 'country_ISO_A2']]

    # Merge population metric from eu_gpd, normalize and merge to country data
    eu_data_pop = eu_data[['iso_a2','pop_est']]
    scaler = MinMaxScaler()
    eu_data_pop.loc[:, ['pop_norm']] = scaler.fit_transform(eu_data_pop[['pop_est']])
    country_data = country_data.merge(eu_data_pop, left_on='country_ISO_A2', right_on='iso_a2')
    country_data['weighted_mean_longdiff'] = country_data['pop_norm'] * country_data['mean_longitudinal_diff_km']

    country_data_sorted = country_data.sort_values('weighted_mean_longdiff', ascending=False)
    source = ColumnDataSource(country_data_sorted)
    columns = [
        TableColumn(field="country_ISO_A2",title="Country Code (ISO_A2)"),
        TableColumn(field="social_timezone", title="Social Timezone"),
        TableColumn(field="weighted_mean_longdiff", title="Weighted (pop.size) avg. dist. to east meridian (km)"),
        TableColumn(field="pop_est", title="Estimated population")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [26]:
def bokeh_sun_table(sun_data):
    source = ColumnDataSource(sun_data)

    # Add data table
    columns = [
        TableColumn(field="iso_a3", title="Country Code (ISO_A3)"),
        TableColumn(field="year", title="Year"),
        TableColumn(field="month", title="Month"),
        TableColumn(field="day", title="Day"),
        TableColumn(field="sunrise_UTC", title="Sunrise (UTC/GMT)"),
        TableColumn(field="sunset_UTC", title="Sunrise (UTC/GMT)")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [46]:
def map_visualization():
    # CREATE MAP  ----------------------------------------------------------------------------------
    # Create Map Panel
    map_pane = pn.pane.Bokeh(sizing_mode='scale_both', width_policy='max')
    start_date = date(2022,1,1)
    end_date = date(2022,12,31)
    selected_date = pn.widgets.DateSlider(name='Date Slider', value=start_date, start=start_date, end=end_date)
    def update_map(event):
        d = selected_date.value
        selected_sundata = sun_data_gpd.query(f'day == {d.day} & month == {d.month} & year == {d.year}')
        map_pane.object = bokeh_plot_map(selected_sundata)
    selected_date.param.watch(update_map, 'value')
    selected_date.param.trigger('value')

    # CREATE DATATABLES ----------------------------------------------------------------------------------
    sizing_dict = dict(sizing_mode='stretch_both', width_policy='auto', margin=10)
    # Create City Table Panel
    country_data_pane = pn.pane.Bokeh(**sizing_dict)
    country_data_pane.object = bokeh_country_table(top_city_data, eu_gpd)

    # Create Sun Table Panel
    sun_data_pane = pn.pane.Bokeh(**sizing_dict)
    sun_data_pane.object = bokeh_sun_table(sun_data_gpd.iloc[:,:-1])

    # Create panel application layout
    map_vis = pn.Column(selected_date, map_pane)
    tabs = pn.Tabs(('Map', map_vis), ('Country Data', country_data_pane), ('Sun Data', sun_data_pane))
    return tabs

app = map_visualization()

In [47]:
app.show()

Launching server at http://localhost:10982
